In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "7"
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
import json
import gc
from tqdm import tqdm
from sklearn.cluster import KMeans
from dataprocess import data_sampler_bert_prompt_deal_first_task_sckd
from model import proto_softmax_layer_bert_prompt
from dataprocess import get_data_loader_bert_prompt
from util import set_seed

In [2]:

class args:
    task = 'fewrel'
    shot = 5

if args.task == 'tacred':
    f = open('config/config_tacred.json','r')
elif args.task == 'fewrel':
    f = open('config/config_fewrel_5and10.json','r')
else:
    raise ValueError('task name is not correct')


config = json.loads(f.read())
f.close()

if args.task == "fewrel":
    config['relation_file'] = "data/fewrel/relation_name.txt"
    config['rel_index'] = "data/fewrel/rel_index.npy"
    config['rel_feature'] = "data/fewrel/rel_feature.npy"
    config['rel_des_file'] = "data/fewrel/relation_description.txt"
    config['num_of_relation'] = 80
    if args.shot == 5:
        config['rel_cluster_label'] = "data/fewrel/CFRLdata_10_100_10_5/rel_cluster_label_0.npy"
        config['training_file'] = "data/fewrel/CFRLdata_10_100_10_5/train_0.txt"
        config['valid_file'] = "data/fewrel/CFRLdata_10_100_10_5/valid_0.txt"
        config['test_file'] = "data/fewrel/CFRLdata_10_100_10_5/test_0.txt"
    elif args.shot == 10:
        config['rel_cluster_label'] = "data/fewrel/CFRLdata_10_100_10_10/rel_cluster_label_0.npy"
        config['training_file'] = "data/fewrel/CFRLdata_10_100_10_10/train_0.txt"
        config['valid_file'] = "data/fewrel/CFRLdata_10_100_10_10/valid_0.txt"
        config['test_file'] = "data/fewrel/CFRLdata_10_100_10_10/test_0.txt"
    else:
        config['rel_cluster_label'] = "data/fewrel/CFRLdata_10_100_10_2/rel_cluster_label_0.npy"
        config['training_file'] = "data/fewrel/CFRLdata_10_100_10_2/train_0.txt"
        config['valid_file'] = "data/fewrel/CFRLdata_10_100_10_2/valid_0.txt"
        config['test_file'] = "data/fewrel/CFRLdata_10_100_10_2/test_0.txt"
else:
    config['relation_file'] = "data/tacred/relation_name.txt"
    config['rel_index'] = "data/tacred/rel_index.npy"
    config['rel_feature'] = "data/tacred/rel_feature.npy"
    config['num_of_relation'] = 41
    if args.shot == 5:
        config['rel_cluster_label'] = "data/tacred/CFRLdata_10_100_10_5/rel_cluster_label_0.npy"
        config['training_file'] = "data/tacred/CFRLdata_10_100_10_5/train_0.txt"
        config['valid_file'] = "data/tacred/CFRLdata_10_100_10_5/valid_0.txt"
        config['test_file'] = "data/tacred/CFRLdata_10_100_10_5/test_0.txt"
    else:
        config['rel_cluster_label'] = "data/tacred/CFRLdata_10_100_10_10/rel_cluster_label_0.npy"
        config['training_file'] = "data/tacred/CFRLdata_10_100_10_10/train_0.txt"
        config['valid_file'] = "data/tacred/CFRLdata_10_100_10_10/valid_0.txt"
        config['test_file'] = "data/tacred/CFRLdata_10_100_10_10/test_0.txt"


config['device'] = torch.device('cuda' if torch.cuda.is_available() and config['use_gpu'] else 'cpu')
config['n_gpu'] = torch.cuda.device_count()
config['batch_size_per_step'] = int(config['batch_size'] / config["gradient_accumulation_steps"])
config['neg_sampling'] = False



donum = 1 
epochs = 1
threshold=0.1

In [3]:
from transformers import BertTokenizer
config['first_task_k-way'] = 10
config['k-shot'] = 5
if config["prompt"] == "hard-complex":
    template = 'the relation between e1 and e2 is mask . '
elif config["prompt"] == "hard-simple":
    template = 'e1 mask e2 . '
else:
    template = None
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sampler = data_sampler_bert_prompt_deal_first_task_sckd(config, tokenizer, template)
#need numpy 1.23.0

/home/thanh/miniconda3/envs/sckd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


final err:  0
[25, [6, 14, 29, 19, 9, 61, 49, 13, 21, 40], [101, 2703, 14262, 2271, 3771, 103, 4649, 6583, 18477, 1012, 1037, 17849, 1997, 2703, 11721, 15916, 20023, 1998, 2767, 1997, 1, 2703, 14262, 2271, 3771, 2, 1010, 2002, 6272, 2000, 1996, 4418, 1997, 3324, 2124, 2004, 1005, 3, 4649, 6583, 18477, 4, 1012, 1005, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2703, 14262, 2271, 3771], [21, 22, 23, 24], [4649, 6583, 18477], [38, 39, 40], 'Q326606', 'Q503708', "a disciple of paul gauguin and friend of paul sérusier , he belonged to the circle of artists known as ' les nabis . '", 45, 1, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

/home/thanh/ConPL/dataprocess.py:2098: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(data)


final err:  0
[12, [58, 66, 74, 80, 14, 71, 68, 12, 57, 13, 48], [101, 5035, 6063, 103, 3539, 2704, 1997, 2710, 1012, 2051, 1998, 3382, 2003, 1037, 2338, 2011, 1, 5035, 6063, 2, 1010, 2280, 3, 3539, 2704, 1997, 2710, 4, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5035, 6063], [17, 18], [3539, 2704, 1997, 2710], [23, 24, 25, 26], 'Q128553', 'Q839078', 'time and chance is a book by kim campbell , former prime minister of canada .', 30, 1, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [4]:
data = []
for steps, (training_data, valid_data, test_data,test_all_data, seen_relations,current_relations) in enumerate(sampler):
    print(steps)
    print(len(training_data))
    print(len(current_relations))
    data.append((training_data, valid_data, test_data, test_all_data, seen_relations, current_relations))
    print("=====================")

0
1000
10
1
50
10
2
50
10
3
50
10
4
50
10
5
50
10
6
50
10
7
50
10


In [5]:
data[0][0][1]

array([25, list([76, 19, 42, 66, 24, 28, 78, 22, 36, 79]),
       list([101, 10488, 16033, 1997, 9610, 2891, 103, 2358, 19419, 9667, 1012, 2002, 2003, 2025, 2000, 2022, 5457, 2007, 1, 10488, 16033, 1997, 9610, 2891, 2, 1010, 1037, 3, 2358, 19419, 9667, 4, 1997, 1996, 3054, 3822, 2301, 4647, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([10488, 16033, 1997, 9610, 2891]), list([19, 20, 21, 22, 23]),
       list([2358, 19419, 9667]), list([28, 29, 30]), 'Q646951', 'Q48235',
       'he is not to be confused with aristo of chios , a stoic philosopher of the mid 3rd century bc .',
       40, 1,
       list([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [7]:
type(test_data)

list